## Oefeningen 2: Transacties

In [1]:
from database import Database
db = Database("../oefeningen.db")

In [ ]:
# Selecteer eerste vijf rijen om inzicht te krijgen in de data
db.query("SELECT * FROM Transacties LIMIT 5")

### Oefening 2.1: Netto Omzet per maand

In [ ]:
# Bereken maand en totaal bedrag per bestelling
db.query("""
SELECT
    STRFTIME('%m', BestelDatum) AS BestelMaand,
    CASE
        WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
        ELSE PrijsExclusief * Aantal * 1.21
    END AS TotaalInclusief
FROM Transacties
""")

In [ ]:
# Gebruik de CTE om te groeperen
db.query("""
WITH OmzetMaand AS (
    SELECT
        STRFTIME('%m', BestelDatum) AS BestelMaand,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
)
SELECT
    BestelMaand,
    SUM(TotaalInclusief) AS OmzetNetto
FROM OmzetMaand
GROUP BY BestelMaand
""")

In [ ]:
# Of zonder subselect (minder leesbaar?)
db.query("""
SELECT
    STRFTIME('%m', BestelDatum) AS BestelMaand,
    SUM(
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END 
    ) AS OmzetNetto
FROM Transacties
GROUP BY STRFTIME('%m', BestelDatum)
""")

### Oefening 2.2: Cumulatieve omzet per dag

In [ ]:
db.query("""
SELECT
    BestelDatum,
    SUM(
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END
    ) AS DagOmzet
FROM Transacties
GROUP BY Besteldatum
""")

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY BestelDatum
)
SELECT
    BestelDatum,
    DagOmzet,
    SUM(DagOmzet) OVER(ORDER BY BestelDatum) AS OmzetCumulatief
FROM OmzetDag
""")

### Oefening 2.3: Rollend gemiddelde 3 dagen

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        COUNT(*) AS TransactiesDag,
        SUM(Aantal) AS ArtikelenDag,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS OmzetDag
    FROM Transacties
    GROUP BY BestelDatum
)
SELECT
    BestelDatum,
    
    TransactiesDag,
    AVG(TransactiesDag) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Transacties3Dagen,
    
    ArtikelenDag,
    AVG(ArtikelenDag) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Artikelen3Dagen,
    
    OmzetDag,
    AVG(OmzetDag) OVER(
        ORDER BY BestelDatum
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS Omzet3Dagen
FROM OmzetDag
""")

### Oefening 2.4: Verschil met vorige dag

In [ ]:
db.query("""
WITH OmzetDag AS (
    SELECT
        BestelDatum,
        SUM(
            CASE
                WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
                ELSE PrijsExclusief * Aantal * 1.21
            END
        ) AS DagOmzet
    FROM Transacties
    GROUP BY BestelDatum
    ORDER BY BestelDatum
)
SELECT
    BestelDatum,
    DagOmzet,
    LAG(DagOmzet) OVER(ORDER BY BestelDatum) AS DagOmzetLagged,
    DagOmzet - LAG(DagOmzet) OVER(ORDER BY BestelDatum) AS OmzetVerschil
FROM OmzetDag
""").head(15)